# Cleaning of leak data

In [1]:
import pandas as pd
from simpledbf import Dbf5
from janitor import clean_names

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [8]:
# Read raw data
FTA_2020_raw = Dbf5("../data/raw/2020_FTA-EJ.dbf").to_dataframe()
FTA_2021_raw = Dbf5("../data/raw/2021_FTA-EJ.dbf").to_dataframe()
FTA_2022_raw = Dbf5("../data/raw/FTA_2022_EJ.dbf").to_dataframe()

In [15]:
print("Shape FTA 20:", FTA_2020_raw.shape)
print("Shape FTA 21:", FTA_2021_raw.shape)
print("Shape FTA 22:", FTA_2022_raw.shape)

Shape FTA 20: (9784, 24)
Shape FTA 21: (10172, 25)
Shape FTA 22: (10064, 25)


In [10]:
print("Total leaks 2020: ", FTA_2020_raw.shape[0])
print("Total leaks 2021: ", FTA_2021_raw.shape[0])
print("Total leaks 2022: ", FTA_2022_raw.shape[0])
print("Total leaks 20-22:", FTA_2020_raw.shape[0] + FTA_2021_raw.shape[0] + FTA_2022_raw.shape[0])

Total leaks 2020:  9784
Total leaks 2021:  10172
Total leaks 2022:  10064
Total leaks 20-22: 30020


## Merge dfs


In [31]:
FTA_clean = clean_names(pd.concat([FTA_2020_raw, FTA_2021_raw, FTA_2022_raw]))
print("Shape of merged df: ", FTA_clean.shape)
FTA_clean.head()

Shape of merged df:  (30020, 25)


,id,fecha_de_c,fecha_de_i,fecha_fina,codunidad,fechaasign,fechalegal,nombreunid,codestado,nombreesta,...,causal,orden,solicitud,cliente,producto,consecruta,codruta,x,y,com1
0,26.0,2019-12-06,2019-12-20,2019-12-20,757.0,2019-12-06,2020-01-06,CT - MANTTO. DE REDES,8.0,Cerrada,...,"EJECUTADA, SE GENERA BA",12773322.0,4495399,101307.0,101307.0,71.0,SECTOR 05/BLOQUE 04 - RUTA 513203,782105.610359,2.423436e+06,NaN
1,35.0,2019-12-17,2019-12-18,2019-12-18,757.0,2019-12-17,2020-01-03,CT - MANTTO. DE REDES,8.0,Cerrada,...,"EJECUTADA, NO GENERA BA",13009880.0,NaN,153367.0,153367.0,213.0,SECTOR 10/BLOQUE 03 - RUTA MO0012,782346.916493,2.419128e+06,NaN
2,36.0,2019-12-17,2019-12-18,2019-12-18,757.0,2019-12-17,2020-01-02,CT - MANTTO. DE REDES,8.0,Cerrada,...,"EJECUTADA, SE GENERA BA",13010131.0,NaN,268357.0,268357.0,229.0,SECTOR 10/BLOQUE 05 - RUTA MO0059,783659.004065,2.420391e+06,NaN
3,42.0,2019-12-16,2019-12-18,2019-12-18,757.0,2019-12-16,2020-01-03,CT - MANTTO. DE REDES,8.0,Cerrada,...,"EJECUTADA, SE GENERA BA",13009404.0,NaN,23043.0,23043.0,105.0,SECTOR 05/BLOQUE 05 - RUTA 512900,781842.636914,2.420929e+06,NaN
4,43.0,2019-12-17,2019-12-18,2019-12-18,757.0,2019-12-17,2020-01-02,CT - MANTTO. DE REDES,8.0,Cerrada,...,"EJECUTADA, SE GENERA BA",13022160.0,NaN,8428.0,8428.0,121.0,SECTOR 05/BLOQUE 08 - RUTA 519410,784277.125146,2.423159e+06,NaN


In [22]:
# NA count
FTA_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          0 non-null      float64
 1   fecha_de_c  0 non-null      object 
 2   fecha_de_i  0 non-null      object 
 3   fecha_fina  0 non-null      object 
 4   codunidad   0 non-null      float64
 5   fechaasign  0 non-null      object 
 6   fechalegal  0 non-null      object 
 7   codestado   0 non-null      float64
 8   nombreesta  0 non-null      object 
 9   codtrabajo  0 non-null      float64
 10  nombretrab  0 non-null      object 
 11  actividad   0 non-null      object 
 12  barrio      0 non-null      object 
 13  com2        0 non-null      object 
 14  causal      0 non-null      object 
 15  orden       0 non-null      float64
 16  cliente     0 non-null      float64
 17  producto    0 non-null      float64
 18  consecruta  0 non-null      float64
 19  codruta     0 non-null      object 
 20  

## Cleaning

In [14]:
FTA_clean.columns

Index(['id', 'fecha_de_i', 'fecha_fina', 'fechaasign', 'fechalegal',
       'nombreunid', 'codestado', 'codtrabajo', 'actividad', 'barrio', 'com2',
       'causal', 'orden', 'solicitud', 'cliente', 'producto', 'consecruta',
       'codruta', 'x', 'y', 'com1'],
      dtype='object')

In [32]:
# Filter FTA that were repaired
FTA_clean = FTA_clean.query("causal == 'EJECUTADA, SE GENERA BA'")

# Drop irrelevant columns
FTA_clean = FTA_clean.drop(["com1", "solicitud", "nombreunid"], axis = 1)

# Sort df by fecha_de_inicio
FTA_clean = FTA_clean.sort_values(by = "fechalegal").reset_index(drop = True)

# Show clean df 
FTA_clean.head() # This df is going to be necessary to define the date of each image to be downloaded

,id,fecha_de_c,fecha_de_i,fecha_fina,codunidad,fechaasign,fechalegal,codestado,nombreesta,codtrabajo,...,barrio,com2,causal,orden,cliente,producto,consecruta,codruta,x,y
0,36.0,2019-12-17,2019-12-18,2019-12-18,757.0,2019-12-17,2020-01-02,8.0,Cerrada,10202.0,...,INFO. MORELOS FRACC.,SE REPARO FUGA EN TOMA ARROYO 1.00X.80...,"EJECUTADA, SE GENERA BA",13010131.0,268357.0,268357.0,229.0,SECTOR 10/BLOQUE 05 - RUTA MO0059,783659.004065,2.420391e+06
1,281.0,2020-01-02,2020-01-02,2020-01-02,757.0,2020-01-02,2020-01-02,8.0,Cerrada,10202.0,...,LOMAS DEL AJEDREZ FRACC.,SE REPARO FUGA EN TOMA ARROYO 1.80X.70,"EJECUTADA, SE GENERA BA",13285314.0,194672.0,194672.0,125.0,SECTOR 10/BLOQUE 04 - RUTA MO0094,783821.977873,2.417947e+06
2,43.0,2019-12-17,2019-12-18,2019-12-18,757.0,2019-12-17,2020-01-02,8.0,Cerrada,10202.0,...,HACIENDAS DE AGS. FRACC.,SE REPARO FUGA EN TOMA BANQUETA 1.80X.70,"EJECUTADA, SE GENERA BA",13022160.0,8428.0,8428.0,121.0,SECTOR 05/BLOQUE 08 - RUTA 519410,784277.125146,2.423159e+06
3,208.0,2019-12-15,2019-12-18,2019-12-18,757.0,2019-12-15,2020-01-02,8.0,Cerrada,10202.0,...,OJOCALIENTE FRACC.,SE REPARO FUGA EN TOMA BANQUETA CONCRETO ...,"EJECUTADA, SE GENERA BA",12981168.0,220903.0,220903.0,14.0,SECTOR 05/BLOQUE 08 - RUTA 514610,783943.411513,2.422147e+06
4,718.0,2020-01-02,2019-12-18,2019-12-18,757.0,2020-01-02,2020-01-02,8.0,Cerrada,10202.0,...,HACIENDAS DE AGS. FRACC.,SE REPARO FUGA EN TOMA BACHE TIERRA TAPADO,"EJECUTADA, SE GENERA BA",13299610.0,159606.0,159606.0,56.0,SECTOR 05/BLOQUE 08 - RUTA 519411,784137.496605,2.423247e+06


In [33]:
FTA_clean.shape

(21725, 22)

In [34]:
FTA_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21725 entries, 0 to 21724
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          21725 non-null  float64
 1   fecha_de_c  21725 non-null  object 
 2   fecha_de_i  21725 non-null  object 
 3   fecha_fina  21725 non-null  object 
 4   codunidad   21725 non-null  float64
 5   fechaasign  21725 non-null  object 
 6   fechalegal  21725 non-null  object 
 7   codestado   21725 non-null  float64
 8   nombreesta  21725 non-null  object 
 9   codtrabajo  21725 non-null  float64
 10  nombretrab  21725 non-null  object 
 11  actividad   21725 non-null  object 
 12  barrio      21725 non-null  object 
 13  com2        21725 non-null  object 
 14  causal      21725 non-null  object 
 15  orden       21725 non-null  float64
 16  cliente     21725 non-null  float64
 17  producto    21725 non-null  float64
 18  consecruta  21725 non-null  float64
 19  codruta     21687 non-nul

In [35]:
# Save clean df 
FTA_clean.to_csv("../data/clean/FTA_clean.csv")